In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Death Star dynamics (Dynamics & Relativity, Sheet 1 Question 4)

In [ ]:
def fun(t, xu, k):
    x, u = xu[:3], xu[3:]
    return [*u, *(k * np.cross(x, u))]

In [ ]:
@interact(
    k=widgets.FloatSlider(min=0, max=30, value=6, continuous_update=False),
    r0=widgets.FloatSlider(min=0, max=4, value=1, continuous_update=False),
    v0_rad=widgets.FloatSlider(min=0, max=4, value=0.5, continuous_update=False),
    v0_perp=widgets.FloatSlider(min=0, max=4, value=2, continuous_update=False),
)
def death_star_demo(k, r0, v0_rad, v0_perp):
    sol = solve_ivp(
        fun, (0, 10), 
        [r0, 0, 0, v0_rad, v0_perp, 0], 
        args=(k,),
#         dense_output=True,
        t_eval=np.arange(0, 10, 0.025)
    )
    assert sol.success, sol.message
    xs, ys, zs = sol.y[0, :], sol.y[1, :], sol.y[2, :]
    us, vs, ws = sol.y[3, :], sol.y[4, :], sol.y[5, :]

    fig = plt.figure(figsize=(14, 8))
    ax = fig.add_subplot(121, projection="3d")
    ax.plot(xs, ys, zs)
    ax.set_xlim((-0.2, 3))
    ax.set_ylim((-0.2, 3))
    ax.set_zlim((-0.2, 3))

    ax = fig.add_subplot(322)
    ax.plot(sol.t, xs, 'k', sol.t, ys, 'r', sol.t, zs, 'b')
    ax.grid()
    ax.legend(('$x$', '$y$', '$z$'))
    ax.set_xscale('log')
    ax.set_yscale('log')
    
    ax = fig.add_subplot(324)
    ax.plot(sol.t, (xs**2 + ys**2 + zs**2)**0.5, 'k')
    ax.legend((r'$|\mathbf{x}|$',))
    ax.grid()
    ax.set_xscale('log')
    ax.set_yscale('log')
    
    ax = fig.add_subplot(326)
    ax.plot(sol.t, us**2 + vs**2 + ws**2, 'k')
    ax.set_ylim((0, 6))
    
    ax.legend((r'$|\dot{\mathbf{x}}|^2$',))

    ax.grid()
    ax.set_xscale('log')
#     ax.set_yscale('log')
    ax.set_xlabel('$t$')
    
    plt.show()